In [15]:
import json
import os
import string
import pandas as pd
import re

In [16]:
vectors = pd.DataFrame()

In [18]:
coelho_files = []
for root, dirs, files in os.walk('TBIBank/Coelho'):
    for filename in files:
        if filename.endswith('txt'):
            coelho_files.append(os.path.join(root, filename))

In [19]:
for file in coelho_files:
    if 'N/Lin' in file:
        senses = {}
        baseline_text = []
        file_match = re.search('n[0-9]{2}', file)
        f = open(f'TBIBank/Coelho/N/{file_match.group(0)}.chstr.cex.txt', 'r')
        for line in f.readlines():
            line = line.strip()
            baseline_text.append(line)
        baseline_text = ''.join(baseline_text)
        senses['text'] = baseline_text
        senses['labels'] = 'Healthy'
        with open(file, 'r') as f:   
            text = json.load(f)
            for relation in text['relations']:
                sense_type = str(relation['Type'])
                if sense_type in senses:
                    senses[sense_type] += 1
                elif sense_type not in senses:
                    senses[sense_type] = 0
                sense = str(relation['Sense'])
                sense = sense.translate(str.maketrans('', '', string.punctuation))
                if sense in senses:
                    senses[sense] +=1
                elif sense not in senses:
                    senses[sense] = 0
                
        sense_df = pd.DataFrame.from_dict([senses])
        vectors = pd.concat([vectors, sense_df], ignore_index=True)

    elif 'TB/Lin' in file:
        senses = {}
        baseline_text = []
        file_match = re.search('tb[0-9]{2}', file)
        f = open(f'TBIBank/Coelho/TB/{file_match.group(0)}.chstr.cex.txt', 'r')
        for line in f.readlines():
            line = line.strip()
            baseline_text.append(line)
        baseline_text = ''.join(baseline_text)
        senses['text'] = baseline_text
        senses['labels'] = 'TBI'
        with open(file, 'r') as f:   
            text = json.load(f)
            for relation in text['relations']:
                sense_type = str(relation['Type'])
                if sense_type in senses:
                    senses[sense_type] += 1
                elif sense_type not in senses:
                    senses[sense_type] = 0
                sense = str(relation['Sense'])
                sense = sense.translate(str.maketrans('', '', string.punctuation))
                if sense in senses:
                    senses[sense] +=1
                elif sense not in senses:
                    senses[sense] = 0
                
        sense_df = pd.DataFrame.from_dict([senses])
        vectors = pd.concat([vectors, sense_df], ignore_index=True)

In [22]:
vectors = vectors.rename(columns={'':'Null'})

In [23]:
vectors = vectors.fillna(0)

In [24]:
vectors['ExpansionConjunction'] = vectors['ExpansionConjunction'].astype('Int64')
vectors['ComparisonConcession'] = vectors['ComparisonConcession'].astype('Int64')
vectors['ContingencyCauseResult'] = vectors['ContingencyCauseResult'].astype('Int64')
vectors['ComparisonContrast'] = vectors['ComparisonContrast'].astype('Int64')
vectors['ExpansionRestatement'] = vectors['ExpansionRestatement'].astype('Int64')	
vectors['EntRel'] = vectors['EntRel'].astype('Int64')
vectors['ContingencyCauseReason'] = vectors['ContingencyCauseReason'].astype('Int64')
vectors['ContingencyCondition']= vectors['ContingencyCondition'].astype('Int64')
vectors['Null']= vectors['Null'].astype('Int64')
vectors['Explicit']= vectors['Explicit'].astype('Int64')
vectors['Implicit']= vectors['Implicit'].astype('Int64')

In [26]:
vectors.to_csv('lin.csv')